In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import random
keras = tf.keras
from keras.utils import np_utils
tfds.disable_progress_bar()
# We need to save this directly to our drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = []
folders = ["rockedge", "paperedge", "scissorsedge", "noneedge"]
for folder in folders:
  dir = f"/content/drive/MyDrive/ME_369P_Project_Data/{folder}"
  for item in os.listdir(dir):
      if item.endswith("jpg"):
        img = cv2.imread(os.path.join(dir, item))
        img = cv2.resize(img, (224, 224))
        data.append([img, folder])
random.shuffle(data)

In [ ]:
# Now that we have our data, we need to convert it into integers based on the map so we can make predictions
CLASS_MAP = {
    "rockedge": 0,
    "paperedge": 1,
    "scissorsedge": 2,
    "noneedge": 3
}

def mapper(val):
    return CLASS_MAP[val]

image, labels = zip(*data)
labels = list(map(mapper, labels))
labels = np_utils.to_categorical(labels)
labels = np.asarray(labels).astype('int32')
image = np.asarray(image)

In [ ]:
start_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

start_model.trainable = False

# define the model
model = keras.Sequential([
        start_model,
        keras.layers.Dropout(0.5),
        keras.layers.Convolution2D(4, (1, 1), padding='valid'),
        keras.layers.Activation('relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Activation('softmax')
    ])
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Now we can train it on our images
history = model.fit(x=image,
    y=labels,
    epochs=15,
    verbose=1)

acc = history.history['accuracy']
print(acc)

Epoch 1/15
100/100 [==============================] - 117s 1s/step - loss: 1.2944 - accuracy: 0.4331
Epoch 2/15
100/100 [==============================] - 118s 1s/step - loss: 0.8712 - accuracy: 0.8119
Epoch 3/15
100/100 [==============================] - 113s 1s/step - loss: 0.5815 - accuracy: 0.9184
Epoch 4/15
100/100 [==============================] - 112s 1s/step - loss: 0.4337 - accuracy: 0.9513
Epoch 5/15
100/100 [==============================] - 114s 1s/step - loss: 0.3463 - accuracy: 0.9622
Epoch 6/15
100/100 [==============================] - 116s 1s/step - loss: 0.2833 - accuracy: 0.9712
Epoch 7/15
100/100 [==============================] - 113s 1s/step - loss: 0.2416 - accuracy: 0.9737
Epoch 8/15
100/100 [==============================] - 118s 1s/step - loss: 0.2106 - accuracy: 0.9794
Epoch 9/15
100/100 [==============================] - 116s 1s/step - loss: 0.1878 - accuracy: 0.9809
Epoch 10/15
100/100 [==============================] - 118s 1s/step - loss: 0.1680 - accura

In [ ]:
model.save("rpsedge2.h5")

In [ ]:
from google.colab import files
files.download("rpsedge2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>